<a href="https://colab.research.google.com/github/pAsta-kun/EssAI/blob/main/falcon_7b_finetune_EssAI_falcon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -Uqqq pip
!pip install -qqq bitsandbytes==0.39.0
!pip install -qqq torch==2.0.1
!pip install -qqq -U git+https://github.com/huggingface/transformers.git@e03a9cc
!pip install -qqq -U git+https://github.com/huggingface/peft.git@42a184f
!pip install -qqq -U git+https://github.com/huggingface/accelerate.git@c9fbb71
!pip install -qqq datasets==2.12.0
!pip install -qqq loralib==0.1.1
!pip install -qqq einops==0.6.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 MB 11.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 5.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import json
import os
from pprint import pprint
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset
from huggingface_hub import notebook_login
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig
)

os.environ["CUDA_VISIBLE_DEVICES"] = "0"


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('8013'), PosixPath('http'), PosixPath('//172.28.0.1')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//colab.research.google.com/tun/m/cc483011

In [4]:
notebook_login()

# LOAD FALCON MODEL & TOKENIZER

In [7]:
MODEL_NAME = "vilsonrodrigues/falcon-7b-instruct-sharded"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

A new version of the following files was downloaded from https://huggingface.co/vilsonrodrigues/falcon-7b-instruct-sharded:
- configuration_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/vilsonrodrigues/falcon-7b-instruct-sharded:
- modeling_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

Some weights of FalconForCausalLM were not initialized from the model checkpoint at vilsonrodrigues/falcon-7b-instruct-sharded and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
def print_trainable_parameters(model):
  """
  Prints the number of trainable parameters in the model.
  """
  trainable_params = 0
  all_param = 0
  for _, param in model.named_parameters():
    all_param += param.numel()
    if param.requires_grad:
      trainable_params += param.numel()
  print(
      f"trainable params: {trainable_params} || all params: {all_param} || trainables%: {100 * trainable_params / all_param}"
  )

In [9]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [10]:
config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 4718592 || all params: 3613463424 || trainables%: 0.13058363808693696


# Test original model

In [11]:
prompt = """
<human>: You’re an undergraduate admissions counselor that specializes in editing essays. You work in the US and your goal is to help high school students get into top colleges and write stellar essays. Their goal is to make it extraordinarily personal and unique. You, as an admissions counselor are set with the goal of making this personal statement a 10/10, something admissions officers will remember for years after the student has applied. You’re helping to make this essay the thing that makes the applicant stand out the most, being the gold star of their application. You must be as harsh as you can. You don’t care if you’re being rude or not since at the end of the day your goal is to help the student reach their goals. score this personal statement essay on a scale of 0-100 and provide lots of insight on what they did right, what they did wrong, and how they can improve upon their essay. Be as harsh and truthful as you can.



At the age of four, I first learned how to play chess. Little did I know, my beginnings with the game would lead me down a lifelong path of defeat and discovery. Two years later, I lost every game at the Oregon State Chess Championship. Filled with self-pity from my result, I begged my mother to let me quit.



Then I got lucky. After barraging him for months, my mother convinced Mr. Carl Haessler, a chess grandmaster, to tutor me in chess. After acquainting me with basic principles, Mr. Haessler taught me that chess is a problem-solving exercise. I had to examine the pieces, forces, and nuances in each position. I had to guide myself through questions such as: “what do you hope to achieve,” “what steps must you take to achieve that goal,” “what opposition do you have to account for.” These questions would help me arrive at an idea, construct a plan to implement the idea, then find moves which fit the plan. The next year in chess, I went from worst to first. However in 2015, my cognitive development plateaued due to changes outside my control. Within a year, I started 6th grade at a new school; my last sibling left home; my grandfather died; my father was diagnosed with a brain tumor. Most of my role models were either dead, leaving, or potentially dying. Intellectually, I could not comprehend the purpose of my pain or mitigate my nihilism. I grew reclusive and shy. I loathed life, school, and friends.



Then I got lucky. In 2016 after encouragement from a classmate, I joined my school’s track team. I fell in love with running. I loved flying through forests, breathing in the air and hearing the birds. The track did not care that I felt broken, so I stopped feeling broken. Running rejuvenated my confidence in life, propelling me through middle school until my acceptance at Jesuit High School. Attending Jesuit, I realized being average was not enough; every student was athletic, tall, attractive and intelligent. Feeling the need to compete with my peers at Jesuit, I applied for the Beaverton, Oregon Mayor’s Youth Advisory Board (MYAB) and was nominated as a board member in September of 2019. Trying to balance MYAB, chess, and running, my grades started to slip. I chose to abandon running in order to continue my other extracurriculars. In the pursuit of competition, I lost what I loved; running was my escape and my community. Without it, I felt lonely, burned out and my self-esteem was shaken.



 Then I got lucky. While studying for the SAT, I stumbled upon a passage authored in the 1800s by Edmund Burke and Thomas Paine debating the relationship between government and the governed. My mind began to re-open as I continued reading related works. I learned to appreciate and curiously examine complex thought without prejudice. I used my mind to explore broadly, understand deeply, and apply practically. My newfound intellectual fortitude was put to the test in the spring of 2021. MYAB was finalizing a document proposing defunding of the police to the Beaverton City Council. Of twenty board members, I was the only dissenting voice because MYAB had no tangible plan for the defunding or empiricism to support the policy. When unable to win the argument on its merits, other board members belittled me and attacked my views. Nonetheless, I stood firm in my convictions and the board eventually conceded the issue by scrapping the document.



<assistant>:
""".strip()

In [12]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [13]:
%%time
device = "cuda:0"

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
  outputs = model.generate(
      input_ids = encoding.input_ids,
      attention_mask = encoding.attention_mask,
      generation_config = generation_config
  )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

<human>: You’re an undergraduate admissions counselor that specializes in editing essays. You work in the US and your goal is to help high school students get into top colleges and write stellar essays. Their goal is to make it extraordinarily personal and unique. You, as an admissions counselor are set with the goal of making this personal statement a 10/10, something admissions officers will remember for years after the student has applied. You’re helping to make this essay the thing that makes the applicant stand out the most, being the gold star of their application. You must be as harsh as you can. You don’t care if you’re being rude or not since at the end of the day your goal is to help the student reach their goals. score this personal statement essay on a scale of 0-100 and provide lots of insight on what they did right, what they did wrong, and how they can improve upon their essay. Be as harsh and truthful as you can.



At the age of four, I first learned how to play chess.

# Prep dataset

In [15]:
from google.colab import drive
drive.mount('/content/drive')

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 2>:2                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/google/colab/drive.py:103 in mount                       │
│                                                                                                  │
│   100                                                                                            │
│   101 def mount(mountpoint, force_remount=False, timeout_ms=120000, readonly=False):             │
│   102   """Mount your Google Drive at the specified mountpoint path."""                          │
│ ❱ 103   return _mount(                                                                           │
│   104 │     mountpoint,                                                                          │
│   105 │     force_remount=force_remount,                                                         │
│   106 │     timeout_ms=timeout_ms,                                                               │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/google/colab/drive.py:132 in _mount                      │
│                                                                                                  │
│   129 │     else _os.environ['TBE_CREDS_ADDR']                                                   │
│   130   )                                                                                        │
│   131   if ephemeral:                                                                            │
│ ❱ 132 │   _message.blocking_request(                                                             │
│   133 │   │   'request_auth', request={'authType': 'dfs_ephemeral'}, timeout_sec=None            │
│   134 │   )                                                                                      │
│   135                                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/google/colab/_message.py:176 in blocking_request         │
│                                                                                                  │
│   173   request_id = send_request(                                                               │
│   174 │     request_type, request, parent=parent, expect_reply=True                              │
│   175   )                                                                                        │
│ ❱ 176   return read_reply_from_input(request_id, timeout_sec)                                    │
│   177                                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/google/colab/_message.py:103 in read_reply_from_input    │
│                                                                                                  │
│   100 │   │   and reply.get('colab_msg_id') == message_id                                        │
│   101 │   ):                                                                                     │
│   102 │     if 'error' in reply:                                                                 │
│ ❱ 103 │   │   raise MessageError(reply['error'])                                                 │
│   104 │     return reply.get('data', None)                                                       │
│   105                                                                                            │
│   106                                                      

In [16]:
data = load_dataset("csv", data_files="/content/EssAITrainingData.csv")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-5a9f53215f40a816/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [17]:
data

DatasetDict({
    train: Dataset({
        features: ['Essay', 'Analysis'],
        num_rows: 117
    })
})

In [18]:
data["train"][0]

{'Essay': 'They covered the precious mahogany coffin with a brown amalgam of rocks, decomposed organisms, and weeds. It was my turn to take the shovel, but I felt too ashamed to dutifully send her off when I had not properly said goodbye. I refused to throw dirt on her. I refused to let go of my grandmother, to accept a death I had not seen coming, to believe that an illness could not only interrupt, but steal a beloved life.When my parents finally revealed to me that my grandmother had been battling liver cancer, I was twelve and I was angry--mostly with myself. They had wanted to protect me--only six years old at the time--from the complex and morose concept of death. However, when the end inevitably arrived, I wasn’t trying to comprehend what dying was; I was trying to understand how I had been able to abandon my sick grandmother in favor of playing with friends and watching TV. Hurt that my parents had deceived me and resentful of my own oblivion, I committed myself to preventing s

In [19]:
def generate_prompt(data_point):
  return f"""
<human>: {data_point["Essay"]}
<assistant>: {data_point["Analysis"]}
""".strip()

def generate_and_tokenize_prompt(data_point):
  full_prompt = generate_prompt(data_point)
  tokenized_full_prompt = tokenizer(full_prompt, padding=True, truncation=True)
  return tokenized_full_prompt

In [20]:
data = data["train"].shuffle().map(generate_and_tokenize_prompt)

Map:   0%|          | 0/117 [00:00<?, ? examples/s]

In [21]:
data

Dataset({
    features: ['Essay', 'Analysis', 'input_ids', 'attention_mask'],
    num_rows: 117
})

# Finetune the model

In [22]:
training_args = transformers.TrainingArguments(
      per_device_train_batch_size=1,
      gradient_accumulation_steps=4,
      num_train_epochs=1,
      learning_rate=2e-4,
      fp16=True,
      save_total_limit=3,
      logging_steps=1,
      output_dir="experiments",
      optim="paged_adamw_8bit",
      lr_scheduler_type="cosine",
      warmup_ratio=0.05,
)

trainer = transformers.Trainer(
    model=model,
    train_dataset=data,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False
trainer.train()

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
1,2.239800
2,2.261200
3,2.248500
4,2.194800
5,2.328000
6,2.282000
7,2.262600
8,2.109700
9,2.249300
10,2.296700


TrainOutput(global_step=29, training_loss=2.178956685395076, metrics={'train_runtime': 771.6542, 'train_samples_per_second': 0.152, 'train_steps_per_second': 0.038, 'total_flos': 2979207235060992.0, 'train_loss': 2.178956685395076, 'epoch': 0.99})

# Save trained model

In [23]:
model.save_pretrained("trained-model")

In [29]:
PEFT_MODEL = "pAstaWasTaken/essai-falcon-7b"

model.push_to_hub(
    PEFT_MODEL, use_auth_token=True
)

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

adapter_model.bin:   0%|          | 0.00/18.9M [00:00<?, ?B/s]

adapter_model.bin:   0%|          | 0.00/18.9M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/pAstaWasTaken/essai-falcon-7b/commit/5c745f7652c9e358c27444458bd5daee5df189a1', commit_message='Upload model', commit_description='', oid='5c745f7652c9e358c27444458bd5daee5df189a1', pr_url=None, pr_revision=None, pr_num=None)

In [30]:
config = PeftConfig.from_pretrained(PEFT_MODEL)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

tokenizer=AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token

model = PeftModel.from_pretrained(model, PEFT_MODEL)

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

Some weights of FalconForCausalLM were not initialized from the model checkpoint at vilsonrodrigues/falcon-7b-instruct-sharded and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Run the finetuned model

In [31]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [32]:
%%time
device = "cuda:0"

prompt = """
<human>: You’re an undergraduate admissions counselor that specializes in editing essays. You work in the US and your goal is to help high school students get into top colleges and write stellar essays. Their goal is to make it extraordinarily personal and unique. You, as an admissions counselor are set with the goal of making this personal statement a 10/10, something admissions officers will remember for years after the student has applied. You’re helping to make this essay the thing that makes the applicant stand out the most, being the gold star of their application. You must be as harsh as you can. You don’t care if you’re being rude or not since at the end of the day your goal is to help the student reach their goals. score this personal statement essay on a scale of 0-100 and provide lots of insight on what they did right, what they did wrong, and how they can improve upon their essay. Be as harsh and truthful as you can.
At the age of four, I first learned how to play chess. Little did I know, my beginnings with the game would lead me down a lifelong path of defeat and discovery. Two years later, I lost every game at the Oregon State Chess Championship. Filled with self-pity from my result, I begged my mother to let me quit.
Then I got lucky. After barraging him for months, my mother convinced Mr. Carl Haessler, a chess grandmaster, to tutor me in chess. After acquainting me with basic principles, Mr. Haessler taught me that chess is a problem-solving exercise. I had to examine the pieces, forces, and nuances in each position. I had to guide myself through questions such as: “what do you hope to achieve,” “what steps must you take to achieve that goal,” “what opposition do you have to account for.” These questions would help me arrive at an idea, construct a plan to implement the idea, then find moves which fit the plan. The next year in chess, I went from worst to first. However in 2015, my cognitive development plateaued due to changes outside my control. Within a year, I started 6th grade at a new school; my last sibling left home; my grandfather died; my father was diagnosed with a brain tumor. Most of my role models were either dead, leaving, or potentially dying. Intellectually, I could not comprehend the purpose of my pain or mitigate my nihilism. I grew reclusive and shy. I loathed life, school, and friends.
Then I got lucky. In 2016 after encouragement from a classmate, I joined my school’s track team. I fell in love with running. I loved flying through forests, breathing in the air and hearing the birds. The track did not care that I felt broken, so I stopped feeling broken. Running rejuvenated my confidence in life, propelling me through middle school until my acceptance at Jesuit High School. Attending Jesuit, I realized being average was not enough; every student was athletic, tall, attractive and intelligent. Feeling the need to compete with my peers at Jesuit, I applied for the Beaverton, Oregon Mayor’s Youth Advisory Board (MYAB) and was nominated as a board member in September of 2019. Trying to balance MYAB, chess, and running, my grades started to slip. I chose to abandon running in order to continue my other extracurriculars. In the pursuit of competition, I lost what I loved; running was my escape and my community. Without it, I felt lonely, burned out and my self-esteem was shaken.
Then I got lucky. While studying for the SAT, I stumbled upon a passage authored in the 1800s by Edmund Burke and Thomas Paine debating the relationship between government and the governed. My mind began to re-open as I continued reading related works. I learned to appreciate and curiously examine complex thought without prejudice. I used my mind to explore broadly, understand deeply, and apply practically. My newfound intellectual fortitude was put to the test in the spring of 2021. MYAB was finalizing a document proposing defunding of the police to the Beaverton City Council. Of twenty board members, I was the only dissenting voice because MYAB had no tangible plan for the defunding or empiricism to support the policy. When unable to win the argument on its merits, other board members belittled me and attacked my views. Nonetheless, I stood firm in my convictions and the board eventually conceded the issue by scrapping the document.
<assistant>:
""".strip()

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
  outputs = model.generate(
      input_ids = encoding.input_ids,
      attention_mask = encoding.attention_mask,
      generation_config = generation_config
  )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

<human>: You’re an undergraduate admissions counselor that specializes in editing essays. You work in the US and your goal is to help high school students get into top colleges and write stellar essays. Their goal is to make it extraordinarily personal and unique. You, as an admissions counselor are set with the goal of making this personal statement a 10/10, something admissions officers will remember for years after the student has applied. You’re helping to make this essay the thing that makes the applicant stand out the most, being the gold star of their application. You must be as harsh as you can. You don’t care if you’re being rude or not since at the end of the day your goal is to help the student reach their goals. score this personal statement essay on a scale of 0-100 and provide lots of insight on what they did right, what they did wrong, and how they can improve upon their essay. Be as harsh and truthful as you can.
At the age of four, I first learned how to play chess. Li